In [2]:
import pandas as pd
import numpy as np


df_path =  '/home/agusitn/Documents/projects/blue-dollar-ml/data/processed/info_finally.csv'
df = pd.read_csv(df_path, parse_dates=['fecha'], index_col='fecha')
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
print(len(df))

df['returns'] = df['valor'].pct_change()

lags = [1,2,3,4,5,10]

for l in lags:
    df[f"lag_{l}"] = df['returns'].shift(l)
print(df.columns)

""" 
Creamos una linea de tendecias en base 5/10/20 dias si la media esperada (rolling_mean ) es positiva y alta
significa que la tendecia del el ultimo mes/semana/dias es que el dolar suba.

Luego si la desviacion estandar es muy volatil quiere decir que el dolar baja y sube constantemente... es decir: 
Si la desviacion estandar (rolling_std) es alta el dolar un dia puede subir un 5% y al otro un 20% y de repente al otro baja un 19%
Si la desviacion estandar es baja el mercado esta estable
"""
windows = [5,10,20]
for w in windows:
    df[f'rolling_mean_{w}'] = df['returns'].rolling(window=w).mean()
    df[f'rolling_std_{w}'] = df['returns'].rolling(window=w).std()

# print(len(df))
# df.dropna(inplace=True)
# print(len(df))
df.to_csv('/home/agusitn/Documents/projects/blue-dollar-ml/data/processed/feature_basic.csv')

print(len(df))

2094
Index(['valor', 'returns', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5',
       'lag_10'],
      dtype='object')
2094


In [4]:
import pandas as pd
import numpy as np

def get_rsi(series_data, periodo=14):
    # series_data ya es la columna de precios. Calculamos directo.
    delta = series_data.diff()
    delta = delta.dropna() # El diff genera un NaN al principio

    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    # Usamos ewm (Exponential Weighted Moving)
    avg_gain = gain.ewm(com=periodo-1, adjust=False).mean()
    avg_loss = loss.ewm(com=periodo-1, adjust=False).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df_path = '/home/agusitn/Documents/projects/blue-dollar-ml/data/processed/info_finally.csv'

df = pd.read_csv(df_path)

df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
df.dropna(subset=['fecha'], inplace=True)

df.drop_duplicates(subset=['fecha'], keep='first', inplace=True)

df.set_index('fecha', inplace=True)

df['valor'] = pd.to_numeric(df['valor'], errors='coerce')
df.dropna(subset=['valor'], inplace=True) # Borramos si falló alguno


print("Calculando RSI...")
df['rsi'] = get_rsi(df['valor'], periodo=14)

print("Calculando Fechas...")
df['day_of_week'] = df.index.dayofweek
df['is_end_of_month'] = (df.index.day > 25).astype(int)

# Limpieza final (el RSI deja los primeros 14 días vacíos)
df.dropna(inplace=True)


save_path = '/home/agusitn/Documents/projects/blue-dollar-ml/data/processed/features_advanced.csv'
df.to_csv(save_path)

print("¡Listo! Archivo features_advanced.csv creado sin errores.")

Calculando RSI...
Calculando Fechas...
¡Listo! Archivo features_advanced.csv creado sin errores.


In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

df_path = '/home/agusitn/Documents/projects/blue-dollar-ml/data/processed/features_advanced.csv'
df = pd.read_csv(df_path, parse_dates=['fecha'], index_col='fecha')

colum_data_leak = [col for col in df.columns if 'rsi' in col]
df[colum_data_leak] = df[colum_data_leak].shift(1)
df.dropna(inplace=True)

df['returns'] = df['valor'].pct_change()
df['target_up'] = (df['returns'] > 0).astype(int)

features = []

for c in df.columns:
    if c not in  ['returns', 'target_up', 'valor']:
        features.append(c)

total_rows = len(df)
train_idx = int(total_rows * 0.70)
val_idx = int(total_rows * 0.85)

x = df[features]
y = df['target_up']

x_train = x.iloc[:train_idx]
y_train = y.iloc[:train_idx]

x_val = x.iloc[train_idx:val_idx]
y_val = y.iloc[train_idx: val_idx]

x_test = x.iloc[val_idx:]
y_test = y.iloc[val_idx:]


""" 
    Debemos normalizar los retornos ya que los datos tienen escalas distintas:
        - El retorno es un número chiquito (ej. 0.01).
        - La volatilidad puede ser otro número diferente.
    La Regresión Logística se confunde si mezclas números grandes con chicos. El StandardScaler pone todo en la misma escala para que el modelo aprenda bien.
"""


scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)

model = LogisticRegression(random_state=42)
model.fit(x_train_scaled, y_train)

predict = model.predict(x_test_scaled)
accuracy = accuracy_score(y_test, predict)

print("\nResultados del Gran Desafío:")
print(f"Modelo con Features Avanzados: {accuracy:.2%}")
print(f"Baseline a vencer (Como Ayer): 58.68%")

if accuracy > 0.5868:
    print("¡OBJETIVO CUMPLIDO! Tu modelo aprendió patrones reales.")
else:
    print("Seguimos empatados o perdiendo. Necesitamos modelos más complejos.")